## Input problem data and imported libs

In [31]:
import sympy as sp
import numpy as np

t = sp.symbols('t')

A = sp.Matrix([[6, -15, 12], [4, -10, 8], [2, -5, 4]])
B = sp.Matrix([1, 0, 1])
F = sp.Matrix([12*t, 0, 12*t])
x0 = sp.Matrix([2, 1, -1])
x1 = sp.Matrix([-13, -9, -9])
T = 1

# A = sp.Matrix([[0, 2], [-2, 0]])
# B = sp.Matrix([1, 2])
# F = sp.Matrix([0, 0])
# x0 = sp.Matrix([5, 0])
# x1 = sp.Matrix([10, 5])
# T = sp.pi

# A = sp.Matrix([[2, -16, -6], [1, -8, -3] , [-2, 16, 6]])
# B = sp.Matrix([1, 0, -2])
# F = sp.Matrix([0, -7*t, -14*t])
# x0 = sp.Matrix([2, 0, -1])
# x1 = sp.Matrix([13, 5, -13])
# T = 1

dim = A.shape[0]

print(f'A: \n{repr(A)}\n')
print(f'B: \n{repr(B)}\n')

A: 
Matrix([
[6, -15, 12],
[4, -10,  8],
[2,  -5,  4]])

B: 
Matrix([
[1],
[0],
[1]])



## Matrix A charpoly and eigenvalues

In [32]:
theta = sp.symbols('theta')
chpoly = A.charpoly(theta).coeffs()
print(chpoly)
eigs = A.eigenvals() 
print(eigs)

[1]
{0: 3}


## Fundamental matrix calculation

In [33]:
eqs = []
omega = sp.symbols('omega')
b = sp.symbols('b0:%d'%dim)
def lamda_power_list(dim) : return np.array([omega ** (i) for i in range(dim)])

for eig, mult in eigs.items():
    lpl = lamda_power_list(dim)
    # print(lpl)
    eq = sp.exp(omega * t) - np.dot(b, lpl)
    eqs.append(eq)
    if mult > 1:
        for i in range(1, mult):
            # lpl = lamda_power_list(lamda, dim, 0)
            # eq = (t ** i) * sp.E ** (lamda * t) - np.dot(b[i:], lpl) 
            eqs.append(sp.diff(eqs[-1], omega)) 

    for i in range(dim):
        eqs[-i] = eqs[-i].subs({omega :eig})

print(f'system: \n{eqs}\n')

b_dict = sp.solve(eqs, b)
print(f'solution: \n{b_dict}\n')
b_val = sp.zeros(rows=1, cols=dim)
for i in range(dim):
    b_val[i] = b_dict[b[i]]

Y = sp.zeros(dim, dim)
for i in range(dim):
    Y += (A ** i) * b_val[i]
print(f'Fundamental matrix: \n{repr(Y)}')


system: 
[1 - b0, -b1 + t, -2*b2 + t**2]

solution: 
{b0: 1, b1: t, b2: t**2/2}

Fundamental matrix: 
Matrix([
[6*t + 1,    -15*t,    12*t],
[    4*t, 1 - 10*t,     8*t],
[    2*t,     -5*t, 4*t + 1]])


## Calculating vector c for program control

In [34]:
Y_inv = Y.inv()
print(f'Y^(-1) :{repr(Y_inv)}')
Q = Y_inv * B
print(f'Q: \n{repr(Q)}\n')
P = sp.integrate(Q * sp.transpose(Q), (t, 0, T))
print(f'P: \n{repr(P)}\n')
N = Y_inv.subs({t : T}) * x1 - x0 - sp.integrate(Y_inv * F, (t, 0, T))
print(f'N: \n{repr(N)}\n')

Y^(-1) :Matrix([
[1 - 6*t,     15*t,   -12*t],
[   -4*t, 10*t + 1,    -8*t],
[   -2*t,      5*t, 1 - 4*t]])
Q: 
Matrix([
[1 - 18*t],
[   -12*t],
[ 1 - 6*t]])

P: 
Matrix([
[91, 66, 25],
[66, 48, 18],
[25, 18,  7]])

N: 
Matrix([
[102],
[ 72],
[ 27]])



In [35]:
c_var = sp.symbols('c0:%d'%dim)
C = sp.Matrix(c_var)

c = sp.zeros(dim, 1)
system_solution = sp.solve(P * C - N, c_var, set=True)[1]
if len(system_solution) != 0:
    c = sp.Matrix(system_solution)
    print(c)
else:
    print('There is no programm control')
    
print(f'transpose(Q) * c: \n{repr(sp.transpose(Q) * c)}')

There is no programm control
transpose(Q) * c: 
Matrix([[0]])
